# Wokflow  **TEST** con Full Bayesiana 

## Inicializacion

In [1]:
# limpio la memoria
Sys.time()
rm(list=ls(all.names=TRUE)) # remove all objects
gc(full=TRUE, verbose=FALSE) # garbage collection

[1] "2025-11-16 18:30:38 UTC"

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,657990,35.2,1454660,77.7,1392631,74.4
Vcells,1221510,9.4,8388608,64.0,1975153,15.1


In [2]:
plocal <- list()

# 506
plocal$qcanaritos <- 5L
plocal$min_data_in_leaf <- 200L
plocal$learning_rate <- 1.0
plocal$gradient_bound <- 0.4


plocal$APO <- 1
plocal$ksemillerio <- 30


In [3]:
PARAM <- list()
PARAM$experimento <- "apo-506-prediccion_agosto_1"
PARAM$semilla_primigenia <- 990013

In [4]:
setwd("/content/buckets/b1/exp")
experimento_folder <- PARAM$experimento
dir.create(experimento_folder, showWarnings=FALSE)
setwd( paste0("/content/buckets/b1/exp/", experimento_folder ))

## Preprocesamiento

### Generacion de la clase_ternaria

In [5]:
Sys.time()
require( "data.table" )

# leo el dataset
dataset <- fread("~/datasets/competencia_02_crudo.csv.gz" )

# calculo el periodo0 consecutivo
dsimple <- dataset[, list(
  "pos" = .I,
  numero_de_cliente,
  periodo0 = as.integer(foto_mes/100)*12 +  foto_mes%%100 )
]


# ordeno
setorder( dsimple, numero_de_cliente, periodo0 )

# calculo topes
periodo_ultimo <- dsimple[, max(periodo0) ]
periodo_anteultimo <- periodo_ultimo - 1


# calculo los leads de orden 1 y 2
dsimple[, c("periodo1", "periodo2") :=
  shift(periodo0, n=1:2, fill=NA, type="lead"),  numero_de_cliente
]

# assign most common class values = "CONTINUA"
dsimple[ periodo0 < periodo_anteultimo, clase_ternaria := "CONTINUA" ]

# calculo BAJA+1
dsimple[ periodo0 < periodo_ultimo &
  ( is.na(periodo1) | periodo0 + 1 < periodo1 ),
  clase_ternaria := "BAJA+1"
]

# calculo BAJA+2
dsimple[ periodo0 < periodo_anteultimo & (periodo0+1 == periodo1 )
  & ( is.na(periodo2) | periodo0 + 2 < periodo2 ),
  clase_ternaria := "BAJA+2"
]

# pego el resultado en el dataset original y grabo
setorder( dsimple, pos )
dataset[, clase_ternaria := dsimple$clase_ternaria ]

rm(dsimple)
gc()
Sys.time()

[1] "2025-11-16 18:30:38 UTC"

Loading required package: data.table



,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,766854,41.0,1454660,77.7,1454660,77.7
Vcells,722144328,5509.6,1017394510,7762.2,845997610,6454.5


[1] "2025-11-16 18:31:05 UTC"

In [6]:
setorder( dataset, foto_mes, clase_ternaria, numero_de_cliente)
dataset[, .N, list(foto_mes, clase_ternaria)]

foto_mes,clase_ternaria,N
<int>,<chr>,<int>
201901,BAJA+1,645
201901,BAJA+2,729
201901,CONTINUA,122899
201902,BAJA+1,733
201902,BAJA+2,707
201902,CONTINUA,123961
201903,BAJA+1,708
201903,BAJA+2,751
201903,CONTINUA,124508


### Eliminacion de Features

Completar a gusto LUEGO de realizar un analisis exploratorio de datos.
<br> No necesariamente en esta Segunda Competencia conviele eliminar los mismos campos que en la Primera ...

### Data Quality

In [7]:
# Salsa Magica Gustavo
dataset[, mprestamos_personales := NULL ]
dataset[, cprestamos_personales := NULL ]

In [8]:
# Salsa Magica?
dataset[, internet := NULL ]
dataset[, tmobile_app := NULL ]
# dataset[, ccheques_depositados_rechazados := NULL ]
# dataset[, mcheques_depositados_rechazados := NULL ]
# dataset[, ccheques_emitidos_rechazados := NULL ]
# dataset[, mcheques_emitidos_rechazados := NULL ]

# dataset[, ccheques_depositados_rechazados := NULL ]
# dataset[, mcheques_depositados_rechazados := NULL ]
# dataset[, ccheques_emitidos_rechazados := NULL ]
# dataset[, mcheques_emitidos_rechazados := NULL ]


In [9]:
cat("Iniciando chequeo y corrección de calidad de datos...\n")

# 1. Definir las columnas numéricas a chequear
cols_to_check <- names(which(sapply(dataset, is.numeric)))
cols_to_check <- setdiff(cols_to_check, c("numero_de_cliente", "foto_mes"))
months <- sort(unique(dataset$foto_mes))

problemas_encontrados <- 0

# Itera por cada mes y cada columna
for (month in months) {
  for (col in cols_to_check) {
    
    # Calcula el rango (min y max) de la columna para ese mes, ignorando NAs
    col_range <- dataset[foto_mes == month, range(get(col), na.rm = TRUE)]
    
    # Si min=0, max=0, y la columna no es "toda NA" (is.finite)
    if (is.finite(col_range[1]) && col_range[1] == 0 && col_range[2] == 0) {
      
      # Informar el problema (solo la primera vez)
      if(problemas_encontrados == 0) cat("Problemas detectados y corregidos:\n")
      cat(paste0("  - Corrigiendo: ", col, " en mes ", month, "\n"))
      
      # Corregir: Reemplazar los 0 de esa columna/mes por NA
      dataset[foto_mes == month & get(col) == 0, (col) := NA_real_]
      
      problemas_encontrados <- problemas_encontrados + 1
    }
  }
}

# 3. Reporte final
if (problemas_encontrados == 0) {
  cat("¡Limpio! No se encontraron columnas para corregir.\n")
} else {
  cat(paste("\nCorrección completada.", problemas_encontrados, "problemas resueltos.\n"))
}

# 4. Limpiar variables temporales
rm(cols_to_check, months, col_range, problemas_encontrados)
cat("--- FIN: Corrección de Calidad de Datos ---\n")

Iniciando chequeo y corrección de calidad de datos...


Warning message in min(x):
“no non-missing arguments to min; returning Inf”
Warning message in max(x):
“no non-missing arguments to max; returning -Inf”
Warning message in min(x):
“no non-missing arguments to min; returning Inf”
Warning message in max(x):
“no non-missing arguments to max; returning -Inf”
Warning message in min(x):
“no non-missing arguments to min; returning Inf”
Warning message in max(x):
“no non-missing arguments to max; returning -Inf”


Problemas detectados y corregidos:
  - Corrigiendo: ctarjeta_visa_debitos_automaticos en mes 201904
  - Corrigiendo: mttarjeta_visa_debitos_automaticos en mes 201904


Warning message in min(x):
“no non-missing arguments to min; returning Inf”
Warning message in max(x):
“no non-missing arguments to max; returning -Inf”


  - Corrigiendo: mrentabilidad en mes 201905
  - Corrigiendo: mrentabilidad_annual en mes 201905
  - Corrigiendo: mcomisiones en mes 201905
  - Corrigiendo: mactivos_margen en mes 201905
  - Corrigiendo: mpasivos_margen en mes 201905
  - Corrigiendo: ccomisiones_otras en mes 201905
  - Corrigiendo: mcomisiones_otras en mes 201905


Warning message in min(x):
“no non-missing arguments to min; returning Inf”
Warning message in max(x):
“no non-missing arguments to max; returning -Inf”
Warning message in min(x):
“no non-missing arguments to min; returning Inf”
Warning message in max(x):
“no non-missing arguments to max; returning -Inf”


  - Corrigiendo: mrentabilidad en mes 201910
  - Corrigiendo: mrentabilidad_annual en mes 201910
  - Corrigiendo: mcomisiones en mes 201910
  - Corrigiendo: mactivos_margen en mes 201910
  - Corrigiendo: mpasivos_margen en mes 201910
  - Corrigiendo: ccajeros_propios_descuentos en mes 201910
  - Corrigiendo: mcajeros_propios_descuentos en mes 201910
  - Corrigiendo: ctarjeta_visa_descuentos en mes 201910
  - Corrigiendo: mtarjeta_visa_descuentos en mes 201910
  - Corrigiendo: ctarjeta_master_descuentos en mes 201910
  - Corrigiendo: mtarjeta_master_descuentos en mes 201910
  - Corrigiendo: ccomisiones_otras en mes 201910
  - Corrigiendo: mcomisiones_otras en mes 201910
  - Corrigiendo: chomebanking_transacciones en mes 201910
  - Corrigiendo: ccajeros_propios_descuentos en mes 202002
  - Corrigiendo: mcajeros_propios_descuentos en mes 202002
  - Corrigiendo: ctarjeta_visa_descuentos en mes 202002
  - Corrigiendo: mtarjeta_visa_descuentos en mes 202002
  - Corrigiendo: ctarjeta_master_d

In [10]:
# library(data.table)
# library(dplyr)
# library(tidyr)

# Sys.time()

# # --- Parámetros ---
# vars_a_analizar <- c("internet")
# umbral <- 0.25

# # --- 0️⃣ Calcular medias por mes ---
# mean_long <- dataset %>%
#   select(foto_mes, all_of(vars_a_analizar)) %>%
#   pivot_longer(
#     cols = all_of(vars_a_analizar),
#     names_to = "variable",
#     values_to = "valor"
#   ) %>%
#   group_by(foto_mes, variable) %>%
#   summarise(mean_value = mean(valor, na.rm = TRUE), .groups = "drop")

# # --- 1️⃣ Encontrar mes de corte ---
# corte_temporal <- mean_long %>%
#   filter(variable %in% vars_a_analizar) %>%
#   filter(!is.na(mean_value)) %>%
#   group_by(variable) %>%
#   summarise(foto_mes_corte = min(foto_mes[mean_value < umbral], na.rm = TRUE), .groups = "drop")

# print(corte_temporal)

# # --- 2️⃣ Transformar dataset ---
# setDT(dataset)
# setorder(dataset, foto_mes)

# for (i in seq_len(nrow(corte_temporal))) {
#   v <- corte_temporal$variable[i]
#   mes_corte <- corte_temporal$foto_mes_corte[i]
  
#   if (is.finite(mes_corte) && !is.na(mes_corte)) {
#     cat("Transformando", v, "desde", mes_corte, "\n")
#     dataset[foto_mes >= mes_corte, (v) := 1 - get(v)]
#   }
# }

# # --- 3️⃣ Verificación ---
# verificacion <- dataset[, lapply(.SD, mean, na.rm = TRUE),
#                          .SDcols = vars_a_analizar,
#                          by = foto_mes]

# print(tail(verificacion))
# Sys.time()

### Aguinaldo

In [11]:
# Se realiza ANTES del Data Drifting para usar valores nominales.

cat("Iniciando FE de Aguinaldo...\n")
setorder(dataset, numero_de_cliente, foto_mes)

# crear el mes (kmes) si no existe
if (!"kmes" %in% colnames(dataset)) {
  dataset[, kmes := foto_mes %% 100]
}

#Calcular ingreso total y lag
#    (Usamos na.fill=0 para que el lag del primer mes sea 0 y no NA)
dataset[, ingreso_total_payroll := mpayroll + mpayroll2]
dataset[, lag_ingreso_total := shift(ingreso_total_payroll, n = 1, type = "lag", fill = 0), by = numero_de_cliente]

# 3. Identificar el aguinaldo
#    Meses típicos de aguinaldo: 6 (Junio), 7 (Julio), 12 (Diciembre)
#    Se asume que el lag_ingreso_total > 0 para evitar divisiones por cero
dataset[, recibe_aguinaldo := 0L]
dataset[
  kmes %in% c(6, 7, 12) &
    lag_ingreso_total > 0 &
    (ingreso_total_payroll / lag_ingreso_total) >= 1.3 &
    (ingreso_total_payroll / lag_ingreso_total) <= 1.8,
  recibe_aguinaldo := 1L
]

# Creo la columna 'mpayroll_normalizado'
#    Se inicializa como el total.
dataset[, mpayroll_normalizado := ingreso_total_payroll]

#    Se normaliza a lo bestia (divide por 1.5) SOLO a los que recibieron aguinaldo
dataset[
  recibe_aguinaldo == 1L,
  mpayroll_normalizado := ingreso_total_payroll / 1.5
]

# Limpieza de columnas originales (como en tu script)
cat("Creando 'mpayroll_normalizado' y eliminando 'mpayroll' y 'mpayroll2'.\n")
dataset[, c("ingreso_total_payroll", "lag_ingreso_total", "mpayroll", "mpayroll2") := NULL]


# 6. Limpiar variables temporales (kmes se usa después, recibe_aguinaldo no)
dataset[, recibe_aguinaldo := NULL]

cat("--- FIN: FE de Aguinaldo ---\n")

Iniciando FE de Aguinaldo...
Creando 'mpayroll_normalizado' y eliminando 'mpayroll' y 'mpayroll2'.
--- FIN: FE de Aguinaldo ---


In [12]:
### Data Drifting (dolar)

In [14]:
# Codigo de experimento
indices_externos <- data.table(
  foto_mes = c(201901, 201902, 201903, 201904, 201905, 201906, 201907, 201908, 201909, 201910, 201911, 201912, 202001, 202002, 202003, 202004, 202005, 202006, 202007, 202008, 202009, 202010, 202011, 202012, 202101, 202102, 202103, 202104, 202105, 202106, 202107, 202108),
  TC_BLUE = c("37,5", "39", "43,65", "46", "46", "43,8", "45,2", "63", "61,25", "69", "69,25", "78,5", "78", "78,5", "83,5", "118", "125", "126", "136", "135", "146", "169", "155", "166", "153", "146", "141", "150", "157", "168", "180,5", "182"),
  TC_OFICIAL = c("38,28", "40,14", "44,4", "45,36", "46,1", "43,7", "45,02", "62,04", "59,82", "63,23", "62,93", "62,99", "63,03", "64,26", "66,58", "69,16", "70,76", "74,07", "76,39", "78,36", "80,63", "83,89", "86,72", "89,87", "92,7", "95,12", "97,69", "98,9", "100,09", "101,17", "102,06", "103,14"),
  TCRM = c("116,5734916", "116,5625812", "119,8157416", "120,2150629", "119,4975567", "114,8101224", "109,8715912", "127,7565287", "130,6961531", "129,7797576", "126,9467229", "123,5951113", "120,8372653", "118,9226283", "114,5040182", "111,5867304", "111,7626924", "116,6307121", "117,9781145", "118,9579865", "119,6485855", "118,1736932", "119,6199112", "123,235859", "122,8366223", "122,3839332", "119,0439564", "116,7635839", "117,2617246", "116,9865438", "113,7702507", "111,9430182"),
  UVA = c("32,03", "32,86", "33,97", "35,42", "36,89", "38,03", "39,2", "40,16", "41,26", "43,43", "45,42", "47,16", "49,05", "50,49", "51,62", "52,95", "54,24", "55,06", "56,09", "57,17", "58,52", "60,16", "61,94", "64,32", "66,54", "69,04", "71,92", "74,87", "78,07", "81,13", "83,82", "86,42"),
  IPC = c("189,6101", "196,7501", "205,9571", "213,0517", "219,5691", "225,537", "230,494", "239,6077", "253,7102", "262,0661", "273,2158", "283,4442", "289,8299", "295,666", "305,5515", "310,1243", "314,9087", "321,9738", "328,2014", "337,0632", "346,6207", "359,657", "371,0211", "385,8826", "401,5071", "415,8595", "435,8657", "453,6503", "468,725", "483,6049", "498,0987", "510,3942"),
  IPIM = c("279", "288,4", "300,2", "314", "329,5", "334,9", "335,2", "372,8", "388,3", "402,3", "424", "439,7", "446,3", "451,3", "455,6", "449,7", "451,3", "467,8", "484,4", "504,2", "522,9", "547,3", "570,1", "595,2", "628,3", "666,5", "692,4", "725,5", "748,8", "772,3", "789,5", "809,4"),
  ripte_indice = c("4042", "4198,76", "4444,6", "4533,03", "4676,25", "4753,19", "4948,27", "5039,93", "5199,08", "5467,59", "5554,15", "5666,48", "6066,07", "6445,13", "6500,72", "6510,18", "6521,87", "6670,93", "6908,52", "6945,86", "7076,47", "7401,81", "7495,03", "7643,41", "7784,1", "8263,33", "8665,19", "9201,59", "9311,61", "9660,13", "10089,96", "10326,11"),
  tasa_i = c("3,71", "3,09", "3,81", "4,45", "4,42", "3,96", "4,14", "4,84", "4,91", "4,06", "3,68", "3,29", "2,83", "2,65", "2,30", "1,54", "2,21", "2,47", "2,43", "2,47", "2,47", "2,65", "2,81", "2,85", "2,85", "2,80", "2,84", "2,84", "2,84", "2,84", "2,84", "2,85"),
  tasa_r = c("0,79", "-0,68", "-0,85", "1,01", "1,28", "1,23", "1,89", "0,81", "-0,94", "0,73", "-0,59", "-0,40", "0,52", "0,64", "-0,97", "0,04", "0,70", "0,27", "0,52", "-0,23", "-0,32", "-1,11", "-0,38", "-1,10", "-1,11", "-0,77", "-1,87", "-1,21", "-0,44", "-0,35", "-0,15", "0,34")
)

# Trabajamos el formato
indices_externos[, foto_mes := as.integer(foto_mes)]

# Reemplazar la coma por el punto en todas las columnas excepto 'foto_mes'
columnas_numericas <- names(indices_externos)[-1]

indices_externos[, (columnas_numericas) := lapply(.SD, function(x) as.numeric(gsub(",", ".", x))), 
                 .SDcols = columnas_numericas]

# convertir a tasa
cols_tasas <- c("tasa_i", "tasa_r")
indices_externos[, (cols_tasas) := lapply(.SD, function(x) x / 100), .SDcols = cols_tasas]

# Verificar la estructura de la tabla
str(indices_externos)

Classes ‘data.table’ and 'data.frame':	32 obs. of  10 variables:
 $ foto_mes    : int  201901 201902 201903 201904 201905 201906 201907 201908 201909 201910 ...
 $ TC_BLUE     : num  37.5 39 43.6 46 46 ...
 $ TC_OFICIAL  : num  38.3 40.1 44.4 45.4 46.1 ...
 $ TCRM        : num  117 117 120 120 119 ...
 $ UVA         : num  32 32.9 34 35.4 36.9 ...
 $ IPC         : num  190 197 206 213 220 ...
 $ IPIM        : num  279 288 300 314 330 ...
 $ ripte_indice: num  4042 4199 4445 4533 4676 ...
 $ tasa_i      : num  0.0371 0.0309 0.0381 0.0445 0.0442 0.0396 0.0414 0.0484 0.0491 0.0406 ...
 $ tasa_r      : num  0.0079 -0.0068 -0.0085 0.0101 0.0128 0.0123 0.0189 0.0081 -0.0094 0.0073 ...
 - attr(*, ".internal.selfref")=<externalptr> 


In [15]:
require("data.table")

#' @title Deflacta columnas monetarias SOBREESCRIBIENDO las originales
#' @description Modifica las columnas monetarias (ej: 'mrentabilidad')
#'              directamente con su valor deflactado.
#'
#' @param dt El data.table principal (ej: 'dataset') que se modificará.
#' @param dt_indices El data.table que contiene todos los índices (ej: 'indices_externos').
#' @param criterio_deflacion El nombre de la columna a usar (ej: "IPC", "TCRM", "tasa_i").
#' @param incluir_dolarizadas TRUE o FALSE. Si es TRUE, ajusta pesos Y dólares.
#'                            Si es FALSE, ajusta SÓLO pesos.
#' @param mes_base Un integer (YYYYMM). Mes base para normalizar (ej: 201901).
#'
#' @return El data.table 'dt' modificado por referencia (no es necesario reasignarlo).

deflactar_dataset <- function(dt, 
                                       dt_indices, 
                                       criterio_deflacion, 
                                       incluir_dolarizadas = FALSE, 
                                       mes_base = 201901) {

  cat(paste0("Iniciando deflación según Criterio: ", criterio_deflacion, "\n"))

  # --- 1. Definir Lógicas y Columnas ---
  
  # Grupo 1: Variables en PESOS
  columnas_pesos <- c(
    "mrentabilidad", "mrentabilidad_annual", "mcomisiones", "mactivos_margen",
    "mpasivos_margen", "mcuentas_saldo", "mcuenta_corriente", "mcaja_ahorro",
    "mcuenta_corriente_adicional", "mcaja_ahorro_adicional",
    "mprestamos_personales", "mprestamos_prendarios", "mprestamos_hipotecarios",
    "mplazo_fijo_pesos", "minversion1_pesos", "minversion2",
    "mpayroll", "mpayroll2", "mcuenta_debitos_automaticos",
    "mtarjeta_visa_consumo", "mtarjeta_master_consumo",
    "mttarjeta_visa_debitos_automaticos", "mttarjeta_master_debitos_automaticos",
    "mpagodeservicios", "mpagomiscuentas", "mtransferencias_recibidas",
    "mtransferencias_emitidas", "mextraccion_autoservicio", "mcheques_depositados",
    "mcheques_emitidos", "mcheques_depositados_rechazados", "mcheques_emitidos_rechazados",
    "matm", "matm_other", "mautoservicio",
    "mcomisiones_mantenimiento", "mcomisiones_otras", "mforex_buy", "mforex_sell",
    "mcajeros_propios_descuentos", "mtarjeta_visa_descuentos", "mtarjeta_master_descuentos",
    "Master_mfinanciacion_limite", "Master_msaldototal", "Master_msaldopesos",
    "Master_mconsumospesos", "Master_mlimitecompra", "Master_madelantopesos",
    "Master_mpagado", "Master_mpagospesos", "Master_mconsumototal", "Master_mpagominimo",
    "Visa_mfinanciacion_limite", "Visa_msaldototal", "Visa_msaldopesos",
    "Visa_mconsumospesos", "Visa_mlimitecompra", "Visa_madelantopesos",
    "Visa_mpagado", "Visa_mpagospesos", "Visa_mconsumototal", "Visa_mpagominimo"
  )
  
  # Grupo 2: Variables DOLARIZADAS
  columnas_dolares <- c(
    "mcaja_ahorro_dolares", "mplazo_fijo_dolares", "minversion1_dolares",
    "Master_msaldodolares", "Master_mconsumosdolares", "Master_madelantodolares",
    "Master_mpagosdolares",
    "Visa_msaldodolares", "Visa_mconsumosdolares", "Visa_madelantodolares",
    "Visa_mpagosdolares"
  )

  # Grupos de lógicas (actualizados a tu tabla final)
  grupo_indices <- c("IPC", "IPIM", "UVA", "ripte_indice") # Se fueron CER e indice_salarios
  grupo_tc <- c("TC_BLUE", "TC_OFICIAL", "TCRM") # Se fue TC_MAYORISTA, TC_MINORISTA
  grupo_tasas <- c("tasa_i", "tasa_r") # Nombres actualizados

  # --- 2. Validar Entradas ---
  if (!criterio_deflacion %in% names(dt_indices)) {
     stop(paste("La columna de criterio '", criterio_deflacion, "' NO existe en 'indices_externos'."))
  }
  
  # --- 3. Preparar el Índice Normalizado ---
  dt_indice_temporal <- dt_indices[, .(foto_mes, INDICE_TEMP = get(criterio_deflacion))]
  setorder(dt_indice_temporal, foto_mes)
  valor_base <- dt_indice_temporal[foto_mes == mes_base, INDICE_TEMP]
  if (length(valor_base) == 0 | is.na(valor_base)) {
    stop(paste("El 'mes_base'", mes_base, "no se encontró o es NA en el índice."))
  }

  if (criterio_deflacion %in% grupo_indices) {
    cat("... Aplicando Lógica de ÍNDICE (Base 1.0)\n")
    dt_indice_temporal[, indice_normalizado := INDICE_TEMP / valor_base]
  } else if (criterio_deflacion %in% grupo_tasas) {
    cat("... Aplicando Lógica de TASA DE INTERÉS\n")
    dt_indice_temporal[, factor := 1 + INDICE_TEMP]
    dt_indice_temporal[, indice_compuesto := cumprod(factor)]
    valor_base_compuesto <- dt_indice_temporal[foto_mes == mes_base, indice_compuesto]
    dt_indice_temporal[, indice_normalizado := indice_compuesto / valor_base_compuesto]
  } else if (criterio_deflacion %in% grupo_tc) {
    cat("... Aplicando Lógica de TIPO DE CAMBIO\n")
    dt_indice_temporal[, indice_normalizado := INDICE_TEMP]
  } else {
    warning(paste("Criterio '", criterio_deflacion, "' no está en un grupo predefinido. Se usará la Lógica de ÍNDICE por defecto."))
    dt_indice_temporal[, indice_normalizado := INDICE_TEMP / valor_base]
  }
  
  dt_indice_para_merge <- dt_indice_temporal[, .(foto_mes, indice_normalizado)]

  # --- 4. Unir el índice al dataset principal (Modifica 'dt' por referencia) ---
  dt[dt_indice_para_merge, on = "foto_mes", indice_normalizado := i.indice_normalizado]

  # --- 5. Seleccionar columnas a deflactar ---
  cols_a_deflactar_final <- columnas_pesos
  if (incluir_dolarizadas) {
    cols_a_deflactar_final <- c(cols_a_deflactar_final, columnas_dolares)
  }
  cols_a_deflactar_final <- intersect(cols_a_deflactar_final, names(dt))
  
  if (length(cols_a_deflactar_final) == 0) {
    cat("Advertencia: Ninguna columna a deflactar fue encontrada. No se hizo nada.\n")
    dt[, indice_normalizado := NULL] # Limpiar la columna de índice
    return(dt)
  }

  # --- 6. Aplicar la deflación (SOBRESCRITURA) ---
  #     (ej: mrentabilidad = mrentabilidad / indice_normalizado)
  dt[, (cols_a_deflactar_final) := lapply(.SD, function(x) {
              x / indice_normalizado
            }), .SDcols = cols_a_deflactar_final]

  # --- 7. Limpiar columna de índice ---
  dt[, indice_normalizado := NULL]

  cat("Deflación completada.", length(cols_a_deflactar_final), "columnas fueron SOBREESCRITAS.\n")
  
  # Limpieza de memoria
  rm(dt_indice_temporal, dt_indice_para_merge, cols_a_deflactar_final, 
     columnas_pesos, columnas_dolares, grupo_indices, grupo_tc, grupo_tasas)
  gc()
  
  return(dt) 
}

In [16]:
# (En la celda de Data Drifting)
deflactar_dataset(
  dt = dataset, 
  dt_indices = indices_externos, 
  criterio_deflacion = "IPC", #IPC , IPIM, UVA, ripte_indice, TC_BLUE, TC_OFICIAL, TCRM, tasa_i, tasa_r
  incluir_dolarizadas = FALSE,  # si es true afecta las columnas en dólares
  mes_base = 201901  #mes al que lleva el valor
)


Iniciando deflación según Criterio: IPC
... Aplicando Lógica de ÍNDICE (Base 1.0)
Deflación completada. 59 columnas fueron SOBREESCRITAS.


In [17]:
#Feature Engineering de Agregación y Ratios Propias ---


# 2.1. Definición de grupos de columnas (exhaustiva)
cols_activos_liquidos <- c("mcuenta_corriente_adicional", "mcuenta_corriente", "mcaja_ahorro", "mcaja_ahorro_adicional", "mcaja_ahorro_dolares", "mplazo_fijo_dolares", "mplazo_fijo_pesos", "minversion1_pesos", "minversion1_dolares", "minversion2")
cols_deuda_tarjetas <- c("Master_msaldototal", "Visa_msaldototal")
cols_ingresos <- c("mtransferencias_recibidas", "mcheques_depositados")
cols_gastos <- c("mcomisiones", "mactivos_margen", "mtarjeta_visa_consumo", "mtarjeta_master_consumo", "mcuenta_debitos_automaticos", "mttarjeta_visa_debitos_automaticos", "mttarjeta_master_debitos_automaticos", "mpagodeservicios", "mpagomiscuentas", "mtransferencias_emitidas", "mextraccion_autoservicio", "mcheques_emitidos", "mforex_buy", "Master_madelantopesos", "Master_madelantodolares", "Visa_madelantopesos", "Visa_madelantodolares")

# 2.2. Creación de variables agregadas
dataset[, total_activos_liquidos := rowSums(.SD, na.rm = TRUE), .SDcols = cols_activos_liquidos]
dataset[, total_deuda_tarjetas := rowSums(.SD, na.rm = TRUE), .SDcols = cols_deuda_tarjetas]
dataset[, total_ingresos := rowSums(.SD, na.rm = TRUE), .SDcols = cols_ingresos]
dataset[, total_gastos := rowSums(.SD, na.rm = TRUE), .SDcols = cols_gastos]

# 2.3. Creación de Ratios de Negocio 
dataset[, ratio_deuda_ingreso := fifelse(total_ingresos > 0, total_deuda_tarjetas / total_ingresos, 0)]
dataset[, ratio_gasto_ingreso := fifelse(total_ingresos > 0, total_gastos / total_ingresos, 0)]
dataset[, ratio_uso_visa := fifelse(Visa_mlimitecompra > 0, mtarjeta_visa_consumo / Visa_mlimitecompra, 0)]
dataset[, ratio_uso_master := fifelse(Master_mlimitecompra > 0, mtarjeta_master_consumo / Master_mlimitecompra, 0)]
dataset[, ratio_fuga_capital := fifelse(mtransferencias_recibidas > 0, mtransferencias_emitidas / mtransferencias_recibidas, 0)]

### Feature Engineering Intra-Mes

Crear variables nuevas a partir de las existentes dentro del mismo registro, **sin** ir a buscar información histórica.
<br> El siguiente código es un mínimo ejemplo, agregar nuevos features a gusto

In [19]:
# el mes 1,2, ..12 , podria servir para detectar estacionalidad
dataset[, kmes := foto_mes %% 100]

# creo un ctr_quarter que tenga en cuenta cuando
# los clientes hace 3 menos meses que estan
# ya que seria injusto considerar las transacciones medidas en menor tiempo
dataset[, ctrx_quarter_normalizado := as.numeric(ctrx_quarter) ]
dataset[cliente_antiguedad == 1, ctrx_quarter_normalizado := ctrx_quarter * 5.0]
dataset[cliente_antiguedad == 2, ctrx_quarter_normalizado := ctrx_quarter * 2.0]
dataset[cliente_antiguedad == 3, ctrx_quarter_normalizado := ctrx_quarter * 1.2]



Sys.time()

[1] "2025-11-16 18:32:33 UTC"

### Feature Engineering Historico

In [20]:
if( !require("Rcpp")) install.packages("Rcpp", repos = "http://cran.us.r-project.org")
require("Rcpp")

Loading required package: Rcpp



In [21]:
# se calculan para los 6 meses previos el minimo, maximo y
#  tendencia calculada con cuadrados minimos
# la formula de calculo de la tendencia puede verse en
#  https://stats.libretexts.org/Bookshelves/Introductory_Statistics/Book%3A_Introductory_Statistics_(Shafer_and_Zhang)/10%3A_Correlation_and_Regression/10.04%3A_The_Least_Squares_Regression_Line
# para la maxíma velocidad esta funcion esta escrita en lenguaje C,
# y no en la porqueria de R o Python

cppFunction("NumericVector fhistC(NumericVector pcolumna, IntegerVector pdesde )
{
  /* Aqui se cargan los valores para la regresion */
  double  x[100] ;
  double  y[100] ;

  int n = pcolumna.size();
  NumericVector out( 5*n );

  for(int i = 0; i < n; i++)
  {
    //lag
    if( pdesde[i]-1 < i )  out[ i + 4*n ]  =  pcolumna[i-1] ;
    else                   out[ i + 4*n ]  =  NA_REAL ;


    int  libre    = 0 ;
    int  xvalor   = 1 ;

    for( int j= pdesde[i]-1;  j<=i; j++ )
    {
       double a = pcolumna[j] ;

       if( !R_IsNA( a ) )
       {
          y[ libre ]= a ;
          x[ libre ]= xvalor ;
          libre++ ;
       }

       xvalor++ ;
    }

    /* Si hay al menos dos valores */
    if( libre > 1 )
    {
      double  xsum  = x[0] ;
      double  ysum  = y[0] ;
      double  xysum = xsum * ysum ;
      double  xxsum = xsum * xsum ;
      double  vmin  = y[0] ;
      double  vmax  = y[0] ;

      for( int h=1; h<libre; h++)
      {
        xsum  += x[h] ;
        ysum  += y[h] ;
        xysum += x[h]*y[h] ;
        xxsum += x[h]*x[h] ;

        if( y[h] < vmin )  vmin = y[h] ;
        if( y[h] > vmax )  vmax = y[h] ;
      }

      out[ i ]  =  (libre*xysum - xsum*ysum)/(libre*xxsum -xsum*xsum) ;
      out[ i + n ]    =  vmin ;
      out[ i + 2*n ]  =  vmax ;
      out[ i + 3*n ]  =  ysum / libre ;
    }
    else
    {
      out[ i       ]  =  NA_REAL ;
      out[ i + n   ]  =  NA_REAL ;
      out[ i + 2*n ]  =  NA_REAL ;
      out[ i + 3*n ]  =  NA_REAL ;
    }
  }

  return  out;
}")

In [22]:
# calcula la tendencia de las variables cols de los ultimos 6 meses
# la tendencia es la pendiente de la recta que ajusta por cuadrados minimos
# La funcionalidad de ratioavg es autoria de  Daiana Sparta,  UAustral  2021

TendenciaYmuchomas <- function(
    dataset, cols, ventana = 6, tendencia = TRUE,
    minimo = TRUE, maximo = TRUE, promedio = TRUE,
    ratioavg = FALSE, ratiomax = FALSE) {
  gc(verbose= FALSE)
  # Esta es la cantidad de meses que utilizo para la historia
  ventana_regresion <- ventana

  last <- nrow(dataset)

  # creo el vector_desde que indica cada ventana
  # de esta forma se acelera el procesamiento ya que lo hago una sola vez
  vector_ids <- dataset[ , numero_de_cliente ]

  vector_desde <- seq(
    -ventana_regresion + 2,
    nrow(dataset) - ventana_regresion + 1
  )

  vector_desde[1:ventana_regresion] <- 1

  for (i in 2:last) {
    if (vector_ids[i - 1] != vector_ids[i]) {
      vector_desde[i] <- i
    }
  }
  for (i in 2:last) {
    if (vector_desde[i] < vector_desde[i - 1]) {
      vector_desde[i] <- vector_desde[i - 1]
    }
  }

  for (campo in cols) {
    nueva_col <- fhistC(dataset[, get(campo)], vector_desde)

    if (tendencia) {
      dataset[, paste0(campo, "_tend", ventana) :=
        nueva_col[(0 * last + 1):(1 * last)]]
    }

    if (minimo) {
      dataset[, paste0(campo, "_min", ventana) :=
        nueva_col[(1 * last + 1):(2 * last)]]
    }

    if (maximo) {
      dataset[, paste0(campo, "_max", ventana) :=
        nueva_col[(2 * last + 1):(3 * last)]]
    }

    if (promedio) {
      dataset[, paste0(campo, "_avg", ventana) :=
        nueva_col[(3 * last + 1):(4 * last)]]
    }

    if (ratioavg) {
      dataset[, paste0(campo, "_ratioavg", ventana) :=
        get(campo) / nueva_col[(3 * last + 1):(4 * last)]]
    }

    if (ratiomax) {
      dataset[, paste0(campo, "_ratiomax", ventana) :=
        get(campo) / nueva_col[(2 * last + 1):(3 * last)]]
    }
  }
}

In [23]:
# Feature Engineering Historico
# Creacion de LAGs
setorder(dataset, numero_de_cliente, foto_mes)

# todo es lagueable, menos la primary key y la clase
cols_lagueables <- copy( setdiff(
  colnames(dataset),
  c("numero_de_cliente", "foto_mes", "clase_ternaria")
))

# https://rdrr.io/cran/data.table/man/shift.html

# lags de orden 1
dataset[,
  paste0(cols_lagueables, "_lag1") := shift(.SD, 1, NA, "lag"),
  by= numero_de_cliente,
  .SDcols= cols_lagueables
]

# lags de orden 2
dataset[,
  paste0(cols_lagueables, "_lag2") := shift(.SD, 2, NA, "lag"),
  by= numero_de_cliente,
  .SDcols= cols_lagueables
]

# agrego los delta lags
for (vcol in cols_lagueables)
{
  dataset[, paste0(vcol, "_delta1") := get(vcol) - get(paste0(vcol, "_lag1"))]
  dataset[, paste0(vcol, "_delta2") := get(vcol) - get(paste0(vcol, "_lag2"))]
}

Sys.time()

[1] "2025-11-16 18:34:36 UTC"

In [24]:
# parametros de Feature Engineering Historico de Tendencias
PARAM$FE_hist$Tendencias$run <- TRUE
PARAM$FE_hist$Tendencias$ventana <- 6
PARAM$FE_hist$Tendencias$tendencia <- TRUE
PARAM$FE_hist$Tendencias$minimo <- FALSE
PARAM$FE_hist$Tendencias$maximo <- FALSE
PARAM$FE_hist$Tendencias$promedio <- FALSE
PARAM$FE_hist$Tendencias$ratioavg <- FALSE
PARAM$FE_hist$Tendencias$ratiomax <- TRUE

In [25]:
# aqui se agregan las tendencias de los ultimos 6 meses

cols_lagueables <- intersect(cols_lagueables, colnames(dataset))
setorder(dataset, numero_de_cliente, foto_mes)

if( PARAM$FE_hist$Tendencias$run) {
    TendenciaYmuchomas(dataset,
    cols = cols_lagueables,
    ventana = PARAM$FE_hist$Tendencias$ventana, # 6 meses de historia
    tendencia = PARAM$FE_hist$Tendencias$tendencia,
    minimo = PARAM$FE_hist$Tendencias$minimo,
    maximo = PARAM$FE_hist$Tendencias$maximo,
    promedio = PARAM$FE_hist$Tendencias$promedio,
    ratioavg = PARAM$FE_hist$Tendencias$ratioavg,
    ratiomax = PARAM$FE_hist$Tendencias$ratiomax
  )
}

ncol(dataset)
Sys.time()

[1] 1109

[1] "2025-11-16 18:36:08 UTC"

### Random Forest

In [26]:
# if( !require("lightgbm")) install.packages("lightgbm")
# require("lightgbm")

In [27]:
# AgregaVarRandomForest <- function() {

#   cat( "inicio AgregaVarRandomForest()\n")
#   gc(verbose= FALSE)
#   dataset[, clase01 := 0L ]
#   dataset[ clase_ternaria %in% c( "BAJA+2", "BAJA+1"),
#       clase01 := 1L ]

#   campos_buenos <- setdiff(
#     colnames(dataset),
#     c( "clase_ternaria", "clase01")
#   )

#   dataset[, entrenamiento :=
#     as.integer( foto_mes %in% PARAM$FE_rf$train$training )]

#   dtrain <- lgb.Dataset(
#     data = data.matrix(dataset[entrenamiento == TRUE, campos_buenos, with = FALSE]),
#     label = dataset[entrenamiento == TRUE, clase01],
#     free_raw_data = FALSE
#   )

#   modelo <- lgb.train(
#      data = dtrain,
#      param = PARAM$FE_rf$lgb_param,
#      verbose = -100
#   )

#   cat( "Fin construccion RandomForest\n" )
#   # grabo el modelo, achivo .model
#   lgb.save(modelo, file="modelo.model" )

#   qarbolitos <- copy(PARAM$FE_rf$lgb_param$num_iterations)

#   periodos <- dataset[ , unique( foto_mes ) ]

#   for( periodo in  periodos )
#   {
#     cat( "periodo = ", periodo, "\n" )
#     datamatrix <- data.matrix(dataset[ foto_mes== periodo, campos_buenos, with = FALSE])

#     cat( "Inicio prediccion\n" )
#     prediccion <- predict(
#         modelo,
#         datamatrix,
#         type = "leaf"
#     )
#     cat( "Fin prediccion\n" )

#     for( arbolito in 1:qarbolitos )
#     {
#        cat( arbolito, " " )
#        hojas_arbol <- unique(prediccion[ , arbolito])

#        for (pos in 1:length(hojas_arbol)) {
#          # el numero de nodo de la hoja, estan salteados
#          nodo_id <- hojas_arbol[pos]
#          dataset[ foto_mes== periodo, paste0(
#             "rf_", sprintf("%03d", arbolito),
#              "_", sprintf("%03d", nodo_id)
#           ) :=  as.integer( nodo_id == prediccion[ , arbolito]) ]

#        }

#        rm( hojas_arbol )
#     }
#     cat( "\n" )

#     rm( prediccion )
#     rm( datamatrix )
#     gc(verbose= FALSE)
#   }

#   gc(verbose= FALSE)

#   # borro clase01 , no debe ensuciar el dataset
#   dataset[ , clase01 := NULL ]

# }

In [28]:
# # Parametros de Feature Engineering  a partir de hojas de Random Forest

# # Estos CUATRO parametros son los que se deben modificar
# PARAM$FE_rf$arbolitos= 20
# PARAM$FE_rf$hojas_por_arbol= 16
# PARAM$FE_rf$datos_por_hoja= 100
# PARAM$FE_rf$mtry_ratio= 0.2

# # Estos son quasi fijos
# PARAM$FE_rf$train$training <- c( 202101, 202102, 202103)

# # Estos TAMBIEN son quasi fijos
# PARAM$FE_rf$lgb_param <-list(
#     # parametros que se pueden cambiar
#     num_iterations = PARAM$FE_rf$arbolitos,
#     num_leaves  = PARAM$FE_rf$hojas_por_arbol,
#     min_data_in_leaf = PARAM$FE_rf$datos_por_hoja,
#     feature_fraction_bynode  = PARAM$FE_rf$mtry_ratio,

#     # para que LightGBM emule Random Forest
#     boosting = "rf",
#     bagging_fraction = ( 1.0 - 1.0/exp(1.0) ),
#     bagging_freq = 1.0,
#     feature_fraction = 1.0,

#     # genericos de LightGBM
#     max_bin = 31L,
#     objective = "binary",
#     first_metric_only = TRUE,
#     boost_from_average = TRUE,
#     feature_pre_filter = FALSE,
#     force_row_wise = TRUE,
#     verbosity = -100,
#     max_depth = -1L,
#     min_gain_to_split = 0.0,
#     min_sum_hessian_in_leaf = 0.001,
#     lambda_l1 = 0.0,
#     lambda_l2 = 0.0,

#     pos_bagging_fraction = 1.0,
#     neg_bagging_fraction = 1.0,
#     is_unbalance = FALSE,
#     scale_pos_weight = 1.0,

#     drop_rate = 0.1,
#     max_drop = 50,
#     skip_drop = 0.5,

#     extra_trees = FALSE
#   )

In [29]:
# # Feature Engineering agregando variables de Random Forest
# #  aqui es donde se hace el trabajo
# AgregaVarRandomForest()

# Sys.time()

In [30]:
ncol(dataset)

[1] 1109

## Modelado

No hay modelado, no se hace optimizacion de hiperparametros.

## Produccion

Las decisiones que se toman para la construccion del modelo final son:
* Los positvos son  POS={"BAJA+1", "BAJA+2"}, esta es una meticulosa decisión.
* Se entrena en los treinta meses del intervalo [201901, 202104]
* Se realiza undersampling al 5%
* Se utilizan los hiperparámetros optimos encontrados en la Bayesian Optimization
   * Se escala min_data_in_leaf

### Final Training Strategy

In [31]:
PARAM$train_final$future <- c(202108)

PARAM$train_final$training <- c(
  201901, 201902, 201903, 201904, 201905, 201906,
  201907, 201908, 201909, 201910, 201911, 201912,
  202001, 202002, 202003, 202004, 202005, 202006,
  202007, 202008, 202009, 202010, 202011, 202012,
  202101, 202102, 202103, 202104, 202105, 202106
)
PARAM$train_final$undersampling <- 0.05  # Silvana y Sofia

In [32]:
# se filtran los meses donde se entrena el modelo final
dataset_train_final <- dataset[foto_mes %in% PARAM$train_final$training]

In [33]:
# canaritos
PARAM$train_final$lgbm$qcanaritos <- plocal$qcanaritos

cols0 <- copy(colnames(dataset_train_final))
filas <- nrow(dataset_train_final)

if( PARAM$train_final$lgbm$qcanaritos > 0 ) {
  for( i in seq(PARAM$train_final$lgbm$qcanaritos) ){
    dataset_train_final[, paste0("canarito_",i) := runif( filas) ]
  }

  # las columnas canaritos mandatoriamente van al comienzo del dataset
  cols_canaritos <- copy( setdiff( colnames(dataset_train_final), cols0 ) )
  setcolorder( dataset_train_final, c( cols_canaritos, cols0 ) )
}

Sys.time()

[1] "2025-11-16 18:36:18 UTC"

#### Registros cambio las proporciones de POS/NEG

In [34]:
# Undersampling, van todos los "BAJA+1" y "BAJA+2" y solo algunos "CONTINIA"

set.seed(PARAM$semilla_primigenia, kind = "L'Ecuyer-CMRG")
dataset_train_final[, azar := runif(nrow(dataset_train_final))]
dataset_train_final[, training := 0L]

dataset_train_final[
  (azar <= PARAM$train_final$undersampling | clase_ternaria %in% c("BAJA+1", "BAJA+2")),
  training := 1L
]

dataset_train_final[, azar:= NULL] # elimino la columna azar

### Target Engineering

In [35]:
# paso la clase a binaria que tome valores {0,1}  enteros
#  BAJA+1 y BAJA+2  son  1,   CONTINUA es 0
#  a partir de ahora ya NO puedo cortar  por prob(BAJA+2) > 1/40

dataset_train_final[,
  clase01 := ifelse(clase_ternaria %in% c("BAJA+2","BAJA+1"), 1L, 0L)
]

### Final Models

Aqui SIEMPRE voy a hacer un semillerio, independientemente de si en la Bayesian Optimization calculé un semillerio en cada iteración.
<br> Entreno un LightGBM para cada semilla,  y guardo el modelo dentro de la carpeta  **modelitos**
<br> Intencionalmente en una primera etapá se generan los modelos y graban, y en una segunda etapa se leen eso modelos y se aplican a los datos del futuro

APO controla cuantas veces se repite el modelo, que se usa para promediar ganancias y reportar en la Pseudo Competencia algo razonable
<br> El modelo puede ser un LightGBM simple (ksemillerio==1)  o un Ensemble Semillerio( ksemillerio > 1 )
<br> Lamentablmente APO necesita utilizar muchas semillas, y eso demanda TIEMPO de corrida

In [36]:

PARAM$train_final$lgbm$param_completo <-  list(
  boosting= "gbdt",
  objective= "binary",
  metric= "custom",
  first_metric_only= FALSE,
  boost_from_average= TRUE,
  feature_pre_filter= FALSE,
  force_row_wise= TRUE,
  verbosity= -100,

  seed= PARAM$semilla_primigenia,

  max_bin= 31L,
  min_data_in_leaf= plocal$min_data_in_leaf,  #este ya es el valor default de LightGBM

  num_iterations= 64L, # dejo libre la cantidad de arboles, zLightGBM se detiene solo
  num_leaves= 9999L, # dejo libre la cantidad de hojas, zLightGBM sabe cuando no hacer un split
  learning_rate= plocal$learning_rate,  # se lo deja en 1.0 para que si el score esta por debajo de gradient_bound no se lo escale
    
  feature_fraction= 0.50, # un valor equilibrado, habra que probar alternativas ...
    
  canaritos= PARAM$train_final$lgbm$qcanaritos, # fundamental en zLightGBM, aqui esta el control del overfitting
  gradient_bound= plocal$gradient_bound   # default de zLightGBM
)

Sys.time()

[1] "2025-11-16 18:36:19 UTC"

In [37]:
# Semillerio Final
PARAM$train_final$APO <- plocal$APO
PARAM$train_final$ksemillerio  <- plocal$ksemillerio

PARAM$train_final$cortes <- c(8000, 8500, 9000, 9500, 10000, 10500, 11000, 11500, 12000)

In [38]:
# detach("package:lightgbm", unload= TRUE)

In [39]:
if( !require("zlightgbm") ) install.packages("https://storage.googleapis.com/open-courses/dmeyf2025-e4a2/zlightgbm_4.6.0.99.tar.gz", repos= NULL, type= "source")
require("zlightgbm")

Loading required package: zlightgbm



In [40]:
if(!require("primes")) install.packages("primes")
require("primes")

Loading required package: primes



In [41]:
primos <- generate_primes(min = 100000, max = 1000000)
set.seed(PARAM$semilla_primigenia, kind = "L'Ecuyer-CMRG")
PARAM$train_final$semillas <- sample(primos)[seq( PARAM$train_final$APO*PARAM$train_final$ksemillerio )]
PARAM$train_final$semillas

[1] 530227 738263 569617 997037 296929 395701 670583 169951 308293 733807
[11] 913513 814367 189823 543877 300647 418007 556753 270527 619303 453797
[21] 154081 367721 112741 230233 189377 779837 833557 481387 628921 449689

In [42]:
campos_buenos <- setdiff(
  colnames(dataset_train_final),
  c( "clase_ternaria", "clase01", "training", "azar")
)

In [43]:
# dejo los datos en formato LightGBM
dtrain_final <- lgb.Dataset(
  data= data.matrix(dataset_train_final[training == 1L, campos_buenos, with= FALSE]),
  label= dataset_train_final[training == 1L, clase01],
  free_raw_data= FALSE
)

cat("filas", nrow(dtrain_final), "columnas", ncol(dtrain_final), "\n")
Sys.time()

filas 256962 columnas 1113 


[1] "2025-11-16 18:36:24 UTC"

In [44]:

# Esta lista guardará la tabla de importancia de CADA uno de los modelos
lista_importancias <- list()

cat("Lista 'lista_importancias' creada para guardar los resultados.\\n")



Lista 'lista_importancias' creada para guardar los resultados.\n

In [45]:
# genero los modelitos
dir.create( "modelitos", showWarnings= FALSE)

param_completo <- copy( PARAM$train_final$lgbm$param_completo)

for( sem in PARAM$train_final$semillas ) {

  arch_modelo <- paste0("./modelitos/mod_", sem, ".txt")
  if( !file.exists( arch_modelo ) )
  {
    param_completo$seed <- sem

    modelito <- lgb.train(
      data= dtrain_final,
      param= param_completo
    )
    
    # 1. Calcular la importancia para ESTE modelo
    impo <- lgb.importance(modelito)
    
    # 2. Añadir la semilla para referencia (opcional, pero útil)
    impo[, semilla := sem]
    
    # 3. Guardar en la lista
    lista_importancias[[length(lista_importancias) + 1]] <- impo
    
    lgb.save( modelito, filename= arch_modelo)
    rm(modelito)
    gc()
  }
}

Sys.time()

[1] "2025-11-16 19:00:57 UTC"

In [46]:
###  Agregar y Promediar Feature Importance

cat("Agregando la importancia de los", length(lista_importancias), "modelos...\\n")

# 1. Combinar todas las tablas de importancia en una sola
tb_importancia_total <- rbindlist(lista_importancias)

# 2. Calcular la Ganancia (Gain) promedio para cada feature
#    Agrupamos por 'Feature' y calculamos la media de 'Gain', 'Cover' y 'Frequency'
tb_importancia_promedio <- tb_importancia_total[, .(
  Gain_Promedio = mean(Gain),
  Cover_Promedio = mean(Cover),
  Frequency_Promedio = mean(Frequency)
), by = Feature]

# 3. Ordenar por la ganancia promedio
setorder(tb_importancia_promedio, -Gain_Promedio)

# 4. Guardar el resultado en un archivo
fwrite(tb_importancia_promedio,
       file = "tb_importancia_promedio.csv", 
       sep = ",")                          

cat("Importancia promedio guardada en 'tb_importancia_promedio.csv'\\n") 


# 5. Mostrar el Top 30 en pantalla
print("--- Top 30 Features (Ganancia Promedio) ---")
print(tb_importancia_promedio[1:30])

# 6. Limpiar memoria
rm(lista_importancias, tb_importancia_total)
gc()



Agregando la importancia de los 30 modelos...\nImportancia promedio guardada en 'tb_importancia_promedio.csv'\n[1] "--- Top 30 Features (Ganancia Promedio) ---"
                                  Feature Gain_Promedio Cover_Promedio
                                   <char>         <num>          <num>
 1:                          total_gastos   0.164673990    0.007960909
 2:              ctrx_quarter_normalizado   0.108629482    0.010403079
 3:                          mcaja_ahorro   0.056303046    0.007747766
 4:                          ctrx_quarter   0.040817006    0.006103229
 5:                total_activos_liquidos   0.028827867    0.006452858
 6:                          cpayroll_trx   0.021081146    0.008812072
 7:                 mtarjeta_visa_consumo   0.018359716    0.006478981
 8:                        mcuentas_saldo   0.015473397    0.011625567
 9:                  mpayroll_normalizado   0.014388196    0.013331874
10:                           Visa_status   0.013695229   

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,1951250,104.3,5503998,294,5503998,294.0
Vcells,8762404547,66851.9,13126328831,100146,9149757857,69807.2


### Scoring

Se hace el predict() del modelo en los datos del futuro

In [47]:
dfuture <- dataset[foto_mes %in% PARAM$train_final$future ]

cols0 <- copy(colnames(dfuture))
filas <- nrow(dfuture)

if( PARAM$train_final$lgbm$qcanaritos > 0 ) {
  for( i in seq(PARAM$train_final$lgbm$qcanaritos) ){
    dfuture[, paste0("canarito_",i) := runif( filas) ]
  }

  # las columnas canaritos mandatoriamente van al comienzo del dataset
  cols_canaritos <- copy( setdiff( colnames(dfuture), cols0 ) )
  setcolorder( dfuture, c( cols_canaritos, cols0 ) )
}

In [48]:
# dataset de future, donde en este caso estoy haciendo testing

mfuture <- data.matrix(dfuture[, campos_buenos, with= FALSE])

# dfuture[, ganancia := ifelse(clase_ternaria=="BAJA+2", 780000, -20000)]

In [49]:
# aplico el modelo a los datos del future

for( vapo in seq(PARAM$train_final$APO) ) {
  # inicializacion en CERO
  vpred_acum <- rep(0.0, nrow(dfuture))
  qacumulados <- 0

  desde <- 1 + (vapo-1)*PARAM$train_final$ksemillerio
  hasta <- desde + PARAM$train_final$ksemillerio - 1
  semillas <- PARAM$train_final$semillas[desde:hasta]

  for( sem in semillas ) {

    arch_modelo <- paste0("./modelitos/mod_", sem, ".txt")
    if( file.exists( arch_modelo ) )
    {
      modelo_final <- lgb.load(arch_modelo) # leo del disco
      # hago el predict() y acumulo
      vpred_acum <- vpred_acum + predict(modelo_final, mfuture)
      qacumulados <- qacumulados + 1
      rm(modelo_final)
      gc()
    }
  }

  if( qacumulados > 0 ) {
    vpred_acum <- vpred_acum / qacumulados  # paso a probabildiad
    
    tb_prediccion <- dfuture[, list(numero_de_cliente, foto_mes)]
    tb_prediccion[, meta_modelo := vapo]
    tb_prediccion[, prob := vpred_acum ]
    setorder( tb_prediccion, -prob )

    # grabo las probabilidades del modelo
    fwrite(tb_prediccion,
      file= "prediccion.txt",
      sep= "\t",
      append= TRUE
    )

    rm(tb_prediccion)
    gc()
  }
}

Sys.time()

[1] "2025-11-16 19:02:02 UTC"

### Clasificacion

Se tomó la decisión de enviar a los 11000 registros con mayor probabilidad de POS={"BAJA+1","BAJA+"}
<br> esto se determinó en forma artesanal analizando meses anterior
<br> esta es una muy importante decisión 

In [50]:
# genero archivos con los  "envios" mejores
dir.create("kaggle", showWarnings=FALSE)

tb_prediccion <- fread("prediccion.txt")

### Subida a Pseudo Kaggle

Aqui viene la verdadera magia de  APO = A Prueba Overfiteres
<br>Se sube un submit a la Pseudo Competencia Kaggle cuya ganancia coincide casi exactamente con la ganancia MEDIA , promediada  APO veces

In [51]:
# --- CÓDIGO DE PRODUCCIÓN FINAL ---

# 1. Definimos el corte óptimo que APRENDIMOS de la validación
corte_optimo_final <- 11200

# 2. Cargamos las predicciones (solo hay meta_modelo = 1)
tb_pred <- tb_prediccion[meta_modelo == 1]

# 3. Aplicamos el corte fijo
tb_pred[, Predicted := 0L] # seteo inicial a 0
tb_pred[1:corte_optimo_final, Predicted := 1L] 

# 4. Creamos el nombre del archivo final
archivo_produccion_kaggle <- paste0("./kaggle/KA", PARAM$experimento, "_FINAL_", corte_optimo_final, ".csv")

# 5. Grabo el archivo
fwrite(tb_pred[, list(numero_de_cliente, Predicted)],
  file= archivo_produccion_kaggle,
  sep= ","
)

cat("Archivo de PRODUCCIÓN FINAL creado en:", archivo_produccion_kaggle, "\n")

Archivo de PRODUCCIÓN FINAL creado en: ./kaggle/KAapo-506-prediccion_agosto_1_FINAL_11200.csv 


In [52]:
Sys.time()

[1] "2025-11-16 19:02:03 UTC"

In [53]:
# Generar archivo para el BOT ---



# 1. Definir un nombre de archivo DISTINTO para el bot
archivo_bot_final <- paste0("./kaggle/BOT_", PARAM$experimento, "_", corte_optimo_final, ".csv")

# 2. Grabar el archivo en el formato requerido
#    - Filtramos los clientes marcados en la celda anterior (Predicted == 1L)
#    - Seleccionamos SOLO la columna numero_de_cliente
fwrite(
  tb_pred[Predicted == 1L, list(numero_de_cliente)],
  file = archivo_bot_final,
  col.names = FALSE
)

cat("El archivo para compartir manualmente es:\\n")
cat(archivo_bot_final, "\\n")

Creando archivo de envío para el Bot @Dos Esfinges...\n¡Éxito! El archivo para compartir manualmente es:\n./kaggle/BOT_apo-506-prediccion_agosto_1_11200.csv \n